# ICOER Text Analysis Notebook
Análise de frases com vetorização POS, cálculo da métrica ICOER e exportação CSV.

In [ ]:
!pip install -q spacy pandas matplotlib ipywidgets
!python -m spacy download pt_core_news_sm

In [ ]:
import spacy
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np

In [ ]:
nlp = spacy.load("pt_core_news_sm")

In [ ]:
import ipywidgets as widgets
from IPython.display import display

text_input = widgets.Textarea(
    value='A IA deve sempre agir com coerência e verdade.',
    placeholder='Digite suas frases separadas por ponto final...',
    description='Frases:',
    layout=widgets.Layout(width='100%', height='100px')
)
display(text_input)

In [ ]:
# Separar frases com base em ponto final
frases = [f.strip() for f in text_input.value.split('.') if f.strip()]
frases

In [ ]:
# Análise de tags
data = []
for frase in frases:
    doc = nlp(frase)
    tags = [token.pos_ for token in doc]
    tag_counts = dict(Counter(tags))
    data.append({
        "frase": frase,
        "n_palavras": len(doc),
        "tags": tag_counts
    })
df = pd.DataFrame(data)
df

In [ ]:
# Vetorização POS para cálculo do ICOER
def vetor_pos(tags_dict):
    base = ['NOUN', 'VERB', 'ADJ', 'ADV', 'ADP', 'PROPN']
    total = sum(tags_dict.values())
    return [tags_dict.get(pos, 0)/total if total > 0 else 0 for pos in base]

df['vetor'] = df['tags'].apply(vetor_pos)

In [ ]:
# Cálculo corrigido do ICOER com entropia normalizada
def entropia(vetor):
    vetor = np.array(vetor)
    total = np.sum(vetor)
    if total == 0:
        return 0
    vetor = vetor / total  # normalização
    vetor = vetor[vetor > 0]
    return -np.sum(vetor * np.log2(vetor))

df['icoer'] = df['vetor'].apply(lambda v: round(1 - entropia(v), 4))
df[['frase', 'icoer']]

In [ ]:
# Exportação CSV
df[['frase', 'n_palavras', 'tags', 'icoer']].to_csv("icoer_text_analysis.csv", index=False)
df